In [2]:
#Load packages
import requests
from bs4 import BeautifulSoup
import os
import json
import pandas as pd
import inspect
import collections
from datetime import datetime as dt
import time
import re
import ast

In [3]:
response = requests.get(url='https://store.steampowered.com/appreviews/413150?json=1').json()

In [5]:
#response

In [6]:
def get_reviews(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json()


In [7]:
def get_n_reviews(appid, n=100):
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'all',
            'language' : 'english',
            'day_range' : 9223372036854775807,
            'review_type' : 'all',
            'purchase_type' : 'all'
            }

    while n > 0:
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 100

        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']

        if len(response['reviews']) < 100: break

    return reviews

In [8]:
#def get_app_id(game_name):
#    response = requests.get(url=f'https://store.steampowered.com/search/?term={game_name}&category1=998', headers={'User-Agent': 'Mozilla/5.0'})
#    soup = BeautifulSoup(response.text, 'html.parser')
#    app_id = soup.find(class_='search_result_row')['data-ds-appid']
#    
#    return app_id

In [9]:
#To get a bunch of app IDs at once
def get_n_appids(n=100, filter_by='topsellers'):
    appids = []
    url = f'https://store.steampowered.com/search/?category1=998&filter={filter_by}&page='
    page = 0

    while page*25 < n:
        page += 1
        response = requests.get(url=url+str(page), headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        for row in soup.find_all(class_='search_result_row'):
            appids.append(row['data-ds-appid'])

    return appids[:n]

In [10]:
#see how long it takes to execute this cell
start = time.time()

reviews = []
appids = get_n_appids(100) #100 app/"game" IDs, 100 top sellers?
for appid in appids:
    reviews += get_n_reviews(appid, 100)


end = time.time()

total_time = end - start
print("\n" + str(total_time))


37.51655387878418


In [11]:
#want to see how reviews looks like.
reviews

#first of all can look up who wrote the review/go to the actual review on Steam by looking up 'steamid'
#might be good to only look at reviews with substantial playtime_forever & playtime_last_two_weeks -> need to parse from "author"
#also 'votes_up' to see how many people found it helpful & votes_funny
#weighted_vote_score - might be good to pull it in to see how it varies

[{'recommendationid': '93757528',
  'author': {'steamid': '76561198049364235',
   'num_games_owned': 147,
   'num_reviews': 2,
   'playtime_forever': 126058,
   'playtime_last_two_weeks': 0,
   'playtime_at_review': 299,
   'last_played': 1672776912},
  'language': 'english',
  'review': "As someone who has played the RU version for 1000hrs+ and have leveled two characters legit and boosted 8 of them (to 50) my two cents about the topic:\n\n+ The game is a fun journey from start to the endgame. The leveling process gets a bit dull, but on your first character it's manageable.\n\n+ The start of the endgame is quite fun. When you first hit 50 there's a lot 'island quest' storylines to complete, reputation quests to unlock, new guardian fights/new dungeons/new raids to experience, some 'sea events' and other 'timed events' to check out. Progression feels pretty good and smooth (until you gear gets around +9, the +10-+15 upgrades are significantly 'harder' to succeed => More fails => More 

In [17]:
df_init = pd.DataFrame(reviews)[['review','author','votes_up','votes_funny','weighted_vote_score', 'voted_up']]


In [18]:
df_init.shape

(9300, 6)

In [19]:
df_init.head()

review  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          As someone who has played the RU version for 1000hrs+ and have leveled two characters legit and boosted 8 of them (to 50) my two cents about the topic:\n\n+ The game is a fun journey from start to the endgame. The leveling process gets a bit dull, but on your first character it's manageable.\n\n+ The start of the endgame is quite fun. When you first hit 50 there's a lot 'island quest' storylines to complete, reputation quests to unlock, new guardian fights/new dungeons/new raids to experience, some 'sea events' and other 'timed events' to check out. Progression feels pretty good and smooth (until you gear gets around +9, the +10-+15 upgrades are significantly 'harder' to succeed => More fails => More costs => Way slower progress).\n\n+ The equalized arena PvP is fun although very CC-heavy and takes a little learning curve to get into. However the rewards are quite poor beyond the 'weekly' ones, so unless you are really into PvP you probably won't spend a lot of time playing it. The ranked PvP is only TRUE SOLO QUEUE, so no premades (which is pretty fun for solo players!).\n\nWhat does this actually have to do with potential P2W and 'forced alts'?\n\n+ Kinda tricked you with the topic. The game is a fun journey even before you need to think about spending.\n\nTo the actual topic:\n\n+ Once you hit your first progression wall (starting from +9 / +10 upgrading) you'll start to see yourself being handicapped by various resources. It might be materials (drops from

In [20]:
#Need to parse out the data from the author field
pd.set_option('display.max_colwidth', None)
print(df_init['author'].head(1)) #example of what this author column looks like.

0    {'steamid': '76561198049364235', 'num_games_owned': 147, 'num_reviews': 2, 'playtime_forever': 126058, 'playtime_last_two_weeks': 0, 'playtime_at_review': 299, 'last_played': 1672776912}
Name: author, dtype: object


### Extracting more data from the author column

In [21]:
#Convert this column to list first
df_init['author'].head()

0       {'steamid': '76561198049364235', 'num_games_owned': 147, 'num_reviews': 2, 'playtime_forever': 126058, 'playtime_last_two_weeks': 0, 'playtime_at_review': 299, 'last_played': 1672776912}
1      {'steamid': '76561199154082086', 'num_games_owned': 9, 'num_reviews': 1, 'playtime_forever': 111232, 'playtime_last_two_weeks': 0, 'playtime_at_review': 106560, 'last_played': 1674117815}
2    {'steamid': '76561199148316490', 'num_games_owned': 203, 'num_reviews': 9, 'playtime_forever': 120182, 'playtime_last_two_weeks': 0, 'playtime_at_review': 120182, 'last_played': 1664724096}
3       {'steamid': '76561198400842440', 'num_games_owned': 44, 'num_reviews': 5, 'playtime_forever': 46943, 'playtime_last_two_weeks': 0, 'playtime_at_review': 46943, 'last_played': 1659067923}
4        {'steamid': '76561198111655942', 'num_games_owned': 373, 'num_reviews': 3, 'playtime_forever': 1040, 'playtime_last_two_weeks': 0, 'playtime_at_review': 1040, 'last_played': 1670828760}
Name: author, dtype: obje

In [22]:
author_fields = df_init['author'].values.tolist()

In [23]:
author_df = pd.DataFrame(author_fields, columns = ['steamid','num_games_owned','num_reviews','playtime_forever',
                                                   'playtime_last_two_weeks','playtime_at_review','last_played'])

In [24]:
author_df.shape

(9300, 7)

In [25]:
df = pd.concat([df_init, author_df], axis=1)

In [26]:
#df.head() #the author columns are included now

In [27]:
df.columns

Index(['review', 'author', 'votes_up', 'votes_funny', 'weighted_vote_score',
       'voted_up', 'steamid', 'num_games_owned', 'num_reviews',
       'playtime_forever', 'playtime_last_two_weeks', 'playtime_at_review',
       'last_played'],
      dtype='object')

In [28]:
#subset and rearrange columns
#won't really need "last_played"
df = df[['steamid','review','num_games_owned','num_reviews','playtime_forever','playtime_last_two_weeks','playtime_at_review',
         'votes_up','votes_funny','weighted_vote_score','voted_up']]

In [29]:
df.head()
#first review is weird.

steamid  \
0  76561198049364235   
1  76561199154082086   
2  76561199148316490   
3  76561198400842440   
4  76561198111655942   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [22]:
#Proportion of reviews with thumbs up or down
df['voted_up'].value_counts()

True     7518
False    1782
Name: voted_up, dtype: int64

In [23]:
#examples of reviews that received a thumbs down
df[df['voted_up']==False]

steamid  \
1     76561199154082086   
2     76561199148316490   
3     76561198400842440   
4     76561198111655942   
5     76561197984114906   
...                 ...   
9294  76561197987360556   
9295  76561197986902042   
9296  76561197977279879   
9297  76561197973315693   
9298  76561198008355893   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

#### Import NLP libraries

In [30]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords

#### Next: lowercase, remove special characters/only keep letters, remove stop words, etc.

In [31]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

#Create a function that tries to clean the reviews
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower() #lowercase
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext) #remove weblinks
    rem_num = re.sub('[0-9]+', '', rem_url) #remove numbers
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')] #remove stop words
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)




In [33]:
#First row looks weird..try taking it out
df = df.iloc[1:]

In [34]:
df.head()

steamid  \
1  76561199154082086   
2  76561199148316490   
3  76561198400842440   
4  76561198111655942   
5  76561197984114906   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

#### Apply the preprocess function

In [ ]:
df['clean_review']=df['review'].map(lambda s:preprocess(s)) 

In [ ]:
#df.head()

In [ ]:
#df['steamid'].nunique()

In [ ]:
#Again, reorder and subset columns
df = df[['clean_review','num_games_owned','num_reviews','playtime_forever','playtime_last_two_weeks','playtime_at_review',
         'votes_up','votes_funny','weighted_vote_score','voted_up']]

In [ ]:
df.head()